# Smart Shopping Assistant using CNN and Feature Matching
**Objective**: Build a simplified Smart Shopping Assistant that:
- Classifies fashion items using a CNN trained on Fashion MNIST.
- Simulates user browsing history/photo gallery.
- Uses basic OpenCV-based feature matching.
- Suggests similar items and filters by price range.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize and reshape data
x_train = x_train / 255.0
x_test = x_test / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Fashion MNIST class names
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train_cat, epochs=3, validation_data=(x_test, y_test_cat))

In [ ]:
# Simulate browsing history with 5 random test images
indices = random.sample(range(len(x_test)), 5)
browsing_images = x_test[indices]
browsing_labels = y_test[indices]

plt.figure(figsize=(10,2))
for i, idx in enumerate(indices):
    plt.subplot(1, 5, i+1)
    plt.imshow(x_test[idx].reshape(28, 28), cmap='gray')
    plt.title(class_names[y_test[idx]])
    plt.axis('off')
plt.show()

In [ ]:
def feature_descriptor(img):
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute((img*255).astype(np.uint8), None)
    return descriptors

# Get descriptors for browsing images
browsing_descriptors = [feature_descriptor(img.reshape(28,28)) for img in browsing_images]

In [ ]:
# Pick a random product from test set
new_idx = random.randint(0, len(x_test) - 1)
new_image = x_test[new_idx]
new_label = y_test[new_idx]
new_desc = feature_descriptor(new_image.reshape(28,28))

# Compare with browsing descriptors using cosine similarity
def match_score(desc1, desc2):
    if desc1 is None or desc2 is None:
        return 0
    min_len = min(len(desc1), len(desc2))
    return cosine_similarity(desc1[:min_len], desc2[:min_len]).mean()

similarities = [match_score(new_desc, desc) for desc in browsing_descriptors]

plt.imshow(new_image.reshape(28,28), cmap='gray')
plt.title(f"New Product: {class_names[new_label]}")
plt.axis('off')
plt.show()

for i, sim in enumerate(similarities):
    print(f"Similarity with browsing item {i+1} ({class_names[browsing_labels[i]]}): {sim:.2f}")

In [ ]:
# Simulate product price dictionary
product_prices = {i: random.randint(20, 200) for i in range(10)}  # class index to price
user_budget = (50, 150)

print("\nSuggested items within your budget:")
for label, price in product_prices.items():
    if user_budget[0] <= price <= user_budget[1]:
        print(f"{class_names[label]} - ${price}")